In [1]:
import tensorflow as tf

sh = [5,2,2,3]
x = tf.ones(sh)
y = tf.ones(sh)
n = x + y
k = x + y + y
ls = [x,n,k,k]

bleh = tf.concat(ls, axis=3)
print ('Shape before argmax: ', bleh.get_shape().as_list())
bleh = tf.argmax(bleh, 3)
print ('Shape after argmax: ', bleh.get_shape().as_list())

z = tf.reduce_mean(ls, 0)
print (z.get_shape().as_list())
p = k.get_shape().as_list()
test_sum = tf.expand_dims(tf.reduce_sum(x, axis=3), 3)
print ("Test sum shape: ", test_sum.get_shape().as_list())

res = tf.concat(ls, axis=3)
mx = tf.reduce_max(res)
print (res.get_shape().as_list())
with tf.Session() as sess:
    print (sess.run(test_sum))
    #print (sess.run(res))
    #print (sess.run(mx))

('Shape before argmax: ', [5, 2, 2, 12])
('Shape after argmax: ', [5, 2, 2])
[5, 2, 2, 3]
('Test sum shape: ', [5, 2, 2, 1])
[5, 2, 2, 12]
[[[[3.]
   [3.]]

  [[3.]
   [3.]]]


 [[[3.]
   [3.]]

  [[3.]
   [3.]]]


 [[[3.]
   [3.]]

  [[3.]
   [3.]]]


 [[[3.]
   [3.]]

  [[3.]
   [3.]]]


 [[[3.]
   [3.]]

  [[3.]
   [3.]]]]


In [2]:
import math
# Code snippets for generating variation ratio

# Following method creates the pixel wise mode tensor.
# Expects input of the type [batch, height, width, num_MC_trials]
def get_mode_tensor(input):
    sh = input.get_shape().as_list()
    batch = sh[0]
    height = sh[1]
    width = sh[2]
    num_mc_trials = sh[3]
    print (batch, height, width, num_mc_trials)
    #res_arr = np.zeros([batch, height, width, 1])
    res_arr = get_multidimensional_list(batch, height, width)
    for i in range(0, batch):
        # reduced_tensor has dimension [height, width, num_MC_trials]
        reduced_tensor = input[i]
        uni_dim_tensor = []
        for x in range(0, height):
            for y in range(0, width):
                uni_dim_tensor = create_uni_dim_tensor(reduced_tensor, x, y)
                y_val, y_indx, y_count = tf.unique_with_counts(uni_dim_tensor)
                res_arr[i][x][y][0] = 1 - tf.cast(tf.reduce_max(y_count), dtype=tf.float32)/num_mc_trials
    return tf.convert_to_tensor(res_arr)
        
    
# Following method expects input of the form [height, width, channels]
def create_uni_dim_tensor(input, height_ind, width_ind):
    res = []
    num_channels = input.get_shape().as_list()[2]
    for i in range(0, num_channels):
        res.append(input[height_ind][width_ind][i])
    return tf.convert_to_tensor(res)


def get_multidimensional_list(batch, height, width):
    ls = [[[[None]*1]*width]*height]*batch
    return ls


def variation_ratio(monte_carlo_samples):
    modified_mc_samples = []
    for sample in monte_carlo_samples:
        modified_mc_samples.append(tf.expand_dims(tf.argmax(sample, 3), 3))

    # Following tensor has dimensions [batch, height, width, num_mc_trials]
    mod_tensor = tf.concat(modified_mc_samples, axis=3)

    return get_mode_tensor(mod_tensor)


# Computing the variation ratio given ls as the input
# Num trials = 4, Batch = 5, height = 2, width = 2, num_classes = 3

sh = [5,2,2,1]
chek = tf.fill(sh, math.exp(-6))
x1 = tf.fill(sh, 0.2)
x2 = tf.fill(sh, 0.3)
x3 = tf.fill(sh, 0.5)
x = tf.concat([x1, x2, x3], axis=3)

y1 = tf.fill(sh, 0.3)
y2 = tf.fill(sh, 0.4)
y3 = tf.fill(sh, 0.3)
y = tf.concat([y1, y2, y3], axis=3)

k1 = tf.fill(sh, 0.5)
k2 = tf.fill(sh, 0.2)
k3 = tf.fill(sh, 0.3)
k = tf.concat([k1, k2, k3], axis=3)

ls = [x,y,k,k]


mode_tensor = variation_ratio(ls)
print (mode_tensor.get_shape().as_list())

with tf.Session() as sess:
    print (sess.run(mode_tensor))
    print (sess.run(chek))

(5, 2, 2, 4)
[5, 2, 2, 1]
[[[[0.5]
   [0.5]]

  [[0.5]
   [0.5]]]


 [[[0.5]
   [0.5]]

  [[0.5]
   [0.5]]]


 [[[0.5]
   [0.5]]

  [[0.5]
   [0.5]]]


 [[[0.5]
   [0.5]]

  [[0.5]
   [0.5]]]


 [[[0.5]
   [0.5]]

  [[0.5]
   [0.5]]]]
[[[[0.00247875]
   [0.00247875]]

  [[0.00247875]
   [0.00247875]]]


 [[[0.00247875]
   [0.00247875]]

  [[0.00247875]
   [0.00247875]]]


 [[[0.00247875]
   [0.00247875]]

  [[0.00247875]
   [0.00247875]]]


 [[[0.00247875]
   [0.00247875]]

  [[0.00247875]
   [0.00247875]]]


 [[[0.00247875]
   [0.00247875]]

  [[0.00247875]
   [0.00247875]]]]


In [3]:
test_x = tf.constant([1.0,2.0,0.5,0.5,0.5,3.0,2.0,5.0,0.6], name='test_x')
y_val, y_idx, y_count = tf.unique_with_counts(test_x)

test_p = test_x / 2
test_q = tf.log(test_x)
with tf.Session() as sess:
    print (sess.run(y_val))
    print (sess.run(y_count))
    print (sess.run(test_p))
    print (sess.run(test_q))

[1.  2.  0.5 3.  5.  0.6]
[1 2 3 1 1 1]
[0.5  1.   0.25 0.25 0.25 1.5  1.   2.5  0.3 ]
[ 0.         0.6931472 -0.6931472 -0.6931472 -0.6931472  1.0986123
  0.6931472  1.609438  -0.5108256]


In [4]:
import math

# Following method computes pixel-wise predictive entropy.
# Expects an input as a list where each element is of the form [batch, height, width, num_classes]
def predictive_entropy(monte_carlo_samples):
    mean_from_trials = tf.reduce_mean(monte_carlo_samples, 0)
    sh = mean_from_trials.get_shape().as_list()
    jitter = tf.fill(sh, math.exp(-6))
    log_mean_from_trials = tf.log(mean_from_trials + jitter)
    product = tf.multiply(mean_from_trials, log_mean_from_trials)
    res = -tf.expand_dims(tf.reduce_sum(product, axis=3), 3)
    return res

pred_entropy = predictive_entropy(ls)
print (pred_entropy.get_shape().as_list())

with tf.Session() as sess:
    print (sess.run(pred_entropy))

[5, 2, 2, 1]
[[[[1.0828611]
   [1.0828611]]

  [[1.0828611]
   [1.0828611]]]


 [[[1.0828611]
   [1.0828611]]

  [[1.0828611]
   [1.0828611]]]


 [[[1.0828611]
   [1.0828611]]

  [[1.0828611]
   [1.0828611]]]


 [[[1.0828611]
   [1.0828611]]

  [[1.0828611]
   [1.0828611]]]


 [[[1.0828611]
   [1.0828611]]

  [[1.0828611]
   [1.0828611]]]]


In [5]:
import math

# Following method computes pixel-wise mutual information.
# Expects an input as a list where each element is of the form [batch, height, width, num_classes]
def mutual_information(monte_carlo_samples):
    processed_monte_carlo_samples = []
    for sample in monte_carlo_samples:
        sh = sample.get_shape().as_list()
        jitter = tf.fill(sh, math.exp(-6))
        processed_monte_carlo_samples.append(tf.multiply(sample, tf.log(sample + jitter)))
    mean_processed_trials = tf.reduce_mean(processed_monte_carlo_samples, 0)
    sum_part = tf.expand_dims(tf.reduce_sum(mean_processed_trials, axis=3), 3)
    return (predictive_entropy(monte_carlo_samples) + sum_part)

mutual_info = mutual_information(ls)
print (mutual_info.get_shape().as_list())

with tf.Session() as sess:
    print (sess.run(mutual_info))

[5, 2, 2, 1]
[[[[0.04580188]
   [0.04580188]]

  [[0.04580188]
   [0.04580188]]]


 [[[0.04580188]
   [0.04580188]]

  [[0.04580188]
   [0.04580188]]]


 [[[0.04580188]
   [0.04580188]]

  [[0.04580188]
   [0.04580188]]]


 [[[0.04580188]
   [0.04580188]]

  [[0.04580188]
   [0.04580188]]]


 [[[0.04580188]
   [0.04580188]]

  [[0.04580188]
   [0.04580188]]]]


In [6]:
sh = [5,2,2,1]
x1 = tf.fill(sh, 0.2)
x2 = tf.fill(sh, 0.3)
x3 = tf.fill(sh, 0.5)
x = tf.concat([x1, x2, x3], axis=3)

# This is used to flatten out the input tensor
p_y = tf.reshape(x, shape=[-1])
print (p_y.get_shape().as_list())

with tf.Session() as sess:
    print (sess.run(x))
    print (sess.run(p_y))

[60]
[[[[0.2 0.3 0.5]
   [0.2 0.3 0.5]]

  [[0.2 0.3 0.5]
   [0.2 0.3 0.5]]]


 [[[0.2 0.3 0.5]
   [0.2 0.3 0.5]]

  [[0.2 0.3 0.5]
   [0.2 0.3 0.5]]]


 [[[0.2 0.3 0.5]
   [0.2 0.3 0.5]]

  [[0.2 0.3 0.5]
   [0.2 0.3 0.5]]]


 [[[0.2 0.3 0.5]
   [0.2 0.3 0.5]]

  [[0.2 0.3 0.5]
   [0.2 0.3 0.5]]]


 [[[0.2 0.3 0.5]
   [0.2 0.3 0.5]]

  [[0.2 0.3 0.5]
   [0.2 0.3 0.5]]]]
[0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5
 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5
 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5
 0.2 0.3 0.5 0.2 0.3 0.5]


In [7]:
shape = p_y.get_shape().as_list()
print (shape)

ones = tf.ones(shape)
ne = tf.to_float(tf.not_equal(ones, p_y))

with tf.Session() as sess:
    print (sess.run(p_y))
    print (sess.run(ones))
    print (sess.run(ne))

[60]
[0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5
 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5
 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5 0.2 0.3 0.5
 0.2 0.3 0.5 0.2 0.3 0.5]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
